In [ ]:
import socket
import threading
import os
import struct

# -------- CONNECT TO SERVER --------
client = socket.socket()
client.connect(("localhost", 9996))
print(">> Connected to secure chat server")

# -------- RECEIVE THREAD --------
def receive():
    while True:
        try:
            data = client.recv(1024)
            if not data:
                continue
            data = data.decode()
            print(data)
        except:
            break

threading.Thread(target=receive, daemon=True).start()

# -------- SEND LOOP --------
while True:
    try:
        choice = input("Send Message (M) or File (F)?: ").strip().lower()

        # ---- MESSAGE ----
        if choice == "m":
            msg = input("You: ")
            client.send(f"MSG:{msg}".encode())
            if msg.lower() == "exit":
                print("Disconnected from server")
                client.close()
                break

        # ---- FILE ----
        elif choice == "f":
            filename = input("Enter filename: ")

            if not os.path.exists(filename):
                print("File not found")
                continue

            filesize = os.path.getsize(filename)

            # Send file header
            client.send(f"FILE:{os.path.basename(filename)}".encode())
            # Send file size
            client.send(struct.pack("!I", filesize))
            # Send file in chunks
            with open(filename, "rb") as f:
                while True:
                    chunk = f.read(4096)
                    if not chunk:
                        break
                    client.send(chunk)

            print("File sent successfully")

        else:
            print("Invalid option")

    except Exception as e:
        print("Send error:", e)
        break


>> Connected to secure chat server


Send Message (M) or File (F)?:  M
You:  hi everyone


('127.0.0.1', 53805) says: send me the file
File received from ('127.0.0.1', 57101): file1.txt
('127.0.0.1', 57101) says: thanku
